In [77]:
#Testing the actual chain
my_blockchain = MinimalBlockChain()
my_blockchain.add_block('OG Block')

print(my_blockchain.blocks)

my_blockchain.add_block('Never Going To Give You Up')
my_blockchain.add_block("Never Going To Let You Down")

print("\nVerify: ")
print(my_blockchain.verify())

#Make a bad block chain
my_blockchain.blocks[1], my_blockchain.blocks[2] = my_blockchain.blocks[2], my_blockchain.blocks[1]

#Verify
print("\n\nEdit chain and verify: \n")
print(my_blockchain.verify())


[<__main__.MinimalBlock object at 0x000001D4FA1F9550>, <__main__.MinimalBlock object at 0x000001D4FA1F9A30>]
Verify: 
True
Edit chain and verify: 
Wrong block index at block 1
Wrong previous hash at block 1
Wrong block index at block 2
Wrong previous hash at block 2
Block 2 backdated
Wrong previous hash at block 3
False


In [69]:
#Python
import hashlib		#Hashing import <3
import datetime

#Individual block
class MinimalBlock:

	#Constructor
    def __init__(self, index, timestamp, payload, prev_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = payload
        self.previous_hash = prev_hash
        self.hash = self.hashVal()
        
        
    #Generate a new hash value using old hash
    def hashVal(self):
        key = hashlib.sha256()
        key.update(str(self.index).encode('utf-8'))
        key.update(str(self.timestamp).encode('utf-8'))
        key.update(str(self.data).encode('utf-8'))
        key.update(str(self.previous_hash).encode('utf-8'))
        return key.hexdigest()	

    #Allow self validation
    def self_validate(self):
        return self.hashVal() == self.hash


In [75]:
#Block chain code
class MinimalBlockChain:

    #Constructor
    def __init__(self):
        self.blocks=[self.get_genesis_block()]
    
    #Get beginning
    def get_genesis_block(self):
        return MinimalBlock(0, datetime.datetime.now(), 'KoolKryptoKurrency', 'LandOfConfusion')

    #Add blocks
    def add_block(self, data):
        self.blocks.append(MinimalBlock(len(self.blocks), datetime.datetime.now(), data, self.blocks[-1].hash))
        
    #Verify
    def verify(self, verbose=True):
        isvalid = True
        
        #Iterate over entire block chain
        for i in range(1, len(self.blocks)):
            
            #Make sure blocks haven't been moved
            if(i != self.blocks[i].index):
                isvalid = False
                if verbose:
                   #Alternately 'Wrong block at index()'.format(i)
                   print(f'Wrong block index at block {i}')
               
            #Self validate
            if not self.blocks[i].self_validate():
                isvalid = False
                if verbose:
                    print(f'Block at index {i} failed to self validate, content changed')
                    
            #What if someone replaces a block with another valid block in the same place
            if(self.blocks[i].previous_hash != self.blocks[i-1].hash):
                isvalid = False
                if verbose:
                    print(f'Wrong previous hash at block {i}')
                    
            #Make sure timestamps are in correct order
            if(self.blocks[i].timestamp<self.blocks[i-1].timestamp):
                isvalid = False
                if verbose:
                    print(f'Block {i} backdated')
        return isvalid